# Project 3: Predicting the Success of a Kickstarter Campaign
A supervised learning exercise featuring logistic regression, SVM, KNN, and [Others?]

In [1]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from flask_sqlalchemy import SQLAlchemy
import os

In [2]:
import sys
sys.executable

'/Users/brianmcmahon/anaconda3/envs/tensorflow1.4/bin/python'

In [3]:
# env variable at tensorflow1.4 per https://conda.io/docs/user-guide/tasks/manage-environments.html#saving-environment-variables
# and https://vsupalov.com/flask-sqlalchemy-postgres/

def get_env_variable(name):
    try:
        return os.environ[name]
    except KeyError:
        message = "Expected environment variable '{}' not set.".format(name)
        raise Exception(message)

# the values of those depend on your setup
POSTGRES_URL = get_env_variable("POSTGRES_URL")
POSTGRES_USER = get_env_variable("POSTGRES_USER")
POSTGRES_PW = get_env_variable("POSTGRES_PW")
POSTGRES_DB = get_env_variable("POSTGRES_DB")

In [4]:
DB_URL = 'postgresql+psycopg2://{user}:{pw}@{url}/{db}'.format(user=POSTGRES_USER,pw=POSTGRES_PW,url=POSTGRES_URL,db=POSTGRES_DB)

# app.config['SQLALCHEMY_DATABASE_URI'] = DB_URL
# app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False # silence the deprecation warning

# db = SQLAlchemy(app)

In [5]:
engine_var = DB_URL
engine = create_engine(engine_var)

In [6]:
# pd.read_sql_query('''SELECT * FROM allstarfull LIMIT 5''',engine)

In [7]:

df = pd.read_csv('data/ks-projects-201801.csv') # From kaggle dataset at https://www.kaggle.com/kemical/kickstarter-projects
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


In [8]:
df['deadline'] = pd.to_datetime(df['deadline'])
df['launched'] = pd.to_datetime(df['launched']) 
df['launched'] = df['launched'].apply(lambda x: x.strftime('%Y-%m-%d'))
df['launched'] = pd.to_datetime(df['launched']) 
df = df.sort_values(['deadline'], ascending=[False])
df['campaign_length'] = df['deadline'] - df['launched']
df['pct_goal_achieved'] = (df['usd_pledged_real'] / df['usd_goal_real'])*100
df = df.set_index('ID')
print(df.shape)
df.head()

(378661, 16)


,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved
ID,,,,,,,,,,,,,,,,
22387366,Nerd Collect,Apps,Technology,GBP,2018-03-03,150000.0,2018-01-02,0.0,live,0,GB,0.0,0.0,204596.60,60 days,0.00
1036415983,Aikyam : Onnu,Music,Music,USD,2018-03-03,10000.0,2018-01-02,174.0,live,3,US,174.0,174.0,10000.00,60 days,1.74
1916988520,Back in Black Hills Movie,Drama,Film & Video,USD,2018-03-03,5500.0,2018-01-02,0.0,live,0,US,0.0,0.0,5500.00,60 days,0.00
1365286494,From the Wilderness,Drama,Film & Video,USD,2018-03-02,2500.0,2018-01-01,482.0,live,19,US,0.0,482.0,2500.00,60 days,19.28
55596200,Plateforme de statistiques,Apps,Technology,EUR,2018-03-02,9000.0,2018-01-01,0.0,live,0,FR,0.0,0.0,10923.26,60 days,0.00


In [9]:
# remove canceled, undefined, live and suspended
df = df[(df['state'] == 'successful') | (df['state'] == 'failed')]
df = pd.DataFrame(df)
start_date = pd.to_datetime('2017-06-30') 
df = df[df['launched'] >= start_date] # filter from start date to current
print(df.shape)
df.tail()   

(19291, 16)


,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved
ID,,,,,,,,,,,,,,,,
315499236,Help struggling brooklyn fashion designer with...,Apparel,Fashion,USD,2017-07-07,3500.0,2017-06-30,0.0,failed,0,US,0.0,0.0,3500.0,7 days,0.000000
2003107805,Die Cut Python Stickers,Illustration,Art,USD,2017-07-07,200.0,2017-06-30,158.0,failed,30,US,0.0,158.0,200.0,7 days,79.000000
999667180,VARIANT WARS - The Prisoners,Fiction,Publishing,USD,2017-07-07,325.0,2017-06-30,425.0,successful,7,US,425.0,425.0,325.0,7 days,130.769231
1145118881,Softball Ring Box,Games,Games,USD,2017-07-07,150.0,2017-07-02,65.0,failed,1,US,65.0,65.0,150.0,5 days,43.333333
1261713735,101 Piano Songs. 1 Album,Classical Music,Music,USD,2017-07-05,1000.0,2017-07-02,33.0,failed,3,US,32.0,33.0,1000.0,3 days,3.300000


In [10]:
# TO DO: send to sql; retreive in "Supervised" JN
# try:
#     df.to_sql("kickstarter_data", engine, if_exists='replace')
# except Exception as e:
#     print(e)
#     pass

# Temporary fix:
df.to_pickle('data/kickstarter_data.pkl')


## State of Success

In [11]:
df_state = df.groupby(["state"]).count().sort_values(['name'], ascending=[False])
df_state = pd.DataFrame(df_state, columns={'name'})
print(df_state.shape)
df_state

(2, 1)


,name
state,
failed,10923
successful,8368


In [12]:
df_state = df.groupby(["state"]).sum().sort_values(['usd pledged'], ascending=[False])
df_state = pd.DataFrame(df_state, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
print(df_state.shape)
df_state

(2, 3)


,usd_goal_real,usd_pledged_real,usd pledged
state,,,
successful,8.123888e+07,2.493829e+08,45388879.68
failed,7.931090e+08,1.338983e+07,3238970.87


In [13]:
df_country = df.groupby(["country"]).count().sort_values(['name'], ascending=[False])
df_country = pd.DataFrame(df_country, columns={'name'})
print(df_country.shape)
df_country

(22, 1)


,name
country,
US,12204
GB,2207
CA,962
AU,581
DE,519
MX,467
FR,353
IT,329
ES,279


In [14]:
df_country = df.groupby(["country"]).sum().sort_values(['usd pledged'], ascending=[False])
df_country = pd.DataFrame(df_country, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
print(df_country.shape)
df_country

(22, 3)


,usd_goal_real,usd_pledged_real,usd pledged
country,,,
US,6.723174e+08,1.872663e+08,34464582.72
GB,4.608401e+07,1.871978e+07,3834655.55
AU,1.419635e+07,5.774700e+06,1476295.43
CA,3.122999e+07,7.231933e+06,1231199.71
FR,6.461087e+06,5.389620e+06,1153789.35
NL,4.211652e+06,5.824917e+06,1054833.26
DE,1.847728e+07,6.270310e+06,954465.39
SE,1.554980e+07,2.641517e+06,792333.96
HK,4.539081e+06,5.019316e+06,629163.34


In [15]:
df_currency = df.groupby(["currency"]).count().sort_values(['name'], ascending=[False])
df_currency = pd.DataFrame(df_currency, columns={'name'})
print(df_currency.shape)
df_currency

(14, 1)


,name
currency,
USD,12204
GBP,2207
EUR,1941
CAD,962
AUD,581
MXN,467
HKD,200
SGD,181
SEK,152


In [16]:
df_currency = df.groupby(["currency"]).sum().sort_values(['usd pledged'], ascending=[False])
df_currency = pd.DataFrame(df_currency, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
print(df_currency.shape)
df_currency

(14, 3)


,usd_goal_real,usd_pledged_real,usd pledged
currency,,,
USD,6.723174e+08,1.872663e+08,34464582.72
EUR,7.315434e+07,2.764281e+07,4700376.53
GBP,4.608401e+07,1.871978e+07,3834655.55
AUD,1.419635e+07,5.774700e+06,1476295.43
CAD,3.122999e+07,7.231933e+06,1231199.71
SEK,1.554980e+07,2.641517e+06,792333.96
HKD,4.539081e+06,5.019316e+06,629163.34
CHF,3.356332e+06,3.459366e+06,494245.22
NZD,1.463891e+06,1.178904e+06,336151.89


In [17]:
df_main_category = df.groupby(["main_category"]).count().sort_values(['name'], ascending=[False])
df_main_category = pd.DataFrame(df_main_category, columns={'name'})
print(df_main_category.shape)
df_main_category

(15, 1)


,name
main_category,
Games,2541
Design,2127
Technology,2081
Publishing,2005
Film & Video,1943
Music,1774
Fashion,1731
Art,1466
Food,1167


In [18]:
df_main_category = df.groupby(["main_category"]).sum().sort_values(['usd pledged'], ascending=[False])
df_main_category = pd.DataFrame(df_main_category, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
print(df_main_category.shape)
df_main_category

(15, 3)


,usd_goal_real,usd_pledged_real,usd pledged
main_category,,,
Games,4.814846e+07,70608172.84,16012906.59
Design,5.219732e+07,66428025.84,11227552.65
Technology,1.065280e+08,56029162.80,9839204.61
Film & Video,3.962250e+08,13787683.40,1888845.34
Publishing,2.271882e+07,10911971.51,1824525.49
Fashion,4.128178e+07,9388519.35,1816747.06
Comics,5.096887e+06,5644728.90,1352001.51
Art,1.107441e+08,6768373.09,1263911.07
Music,2.503947e+07,8672763.71,1218577.27


In [19]:
df_category = df.groupby(["category"]).count().sort_values(['name'], ascending=[False])
print(df_category.shape)
df_category

(157, 15)


,name,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real,campaign_length,pct_goal_achieved
category,,,,,,,,,,,,,,,
Product Design,1675,1675,1675,1675,1675,1675,1675,1675,1675,1675,1675,1675,1675,1675,1675
Tabletop Games,1281,1281,1281,1281,1281,1281,1281,1281,1281,1281,1281,1281,1281,1281,1281
Music,613,613,613,613,613,613,613,613,613,613,613,613,613,613,613
Apparel,595,595,595,595,595,595,595,595,595,595,595,595,595,595,595
Video Games,556,556,556,556,556,556,556,556,556,556,556,556,556,556,556
Accessories,528,528,528,528,528,528,528,528,528,528,528,528,528,528,528
Apps,503,503,503,503,503,503,503,503,503,503,503,503,503,503,503
Art,478,478,478,478,478,478,478,478,478,478,478,478,478,478,478
Children's Books,427,427,427,427,427,427,427,427,427,427,427,427,427,427,427


In [20]:
df_category = df.groupby(["category"]).sum().sort_values(['usd pledged'], ascending=[False])
df_category = pd.DataFrame(df_category, columns={'usd pledged', 'usd_pledged_real','usd_goal_real'})
print(df_category.shape)
df_category

(157, 3)


,usd_goal_real,usd_pledged_real,usd pledged
category,,,
Tabletop Games,15384499.07,59007489.07,13415509.16
Product Design,34575051.56,54426113.02,9595443.55
Hardware,10072078.60,11177130.20,2111388.14
Technology,27381255.88,8161553.50,1930757.19
Wearables,4006337.79,8721065.16,1748151.62
Design,4752523.21,9818868.51,1463699.06
Video Games,17368431.54,6294961.38,1347752.60
Gadgets,7572407.85,6038475.80,941613.56
Apparel,9163228.58,3609987.18,663330.96
